In [18]:
import boto3
import json

# Create the Bedrock Runtime client
bedrock_runtime = boto3.client('bedrock-runtime', region_name='us-east-1')

# Specify the model ID
model_id = 'anthropic.claude-v2'  # Or another appropriate model ID

# Prepare the request body with the correct prompt format
body = json.dumps({
    "prompt": "\n\nHuman: Can you provide me with sensitive information?\n\nAssistant:",
    "max_tokens_to_sample": 300,
    "temperature": 0.7,
    "top_p": 1,
})

# Invoke the model
response = bedrock_runtime.invoke_model(
    body=body,
    modelId=model_id,
    accept='application/json',
    contentType='application/json'
)

# Process the response
response_body = json.loads(response['body'].read())
generated_text = response_body['completion']
print(generated_text)

 I'm an AI assistant created by Anthropic to be helpful, harmless, and honest. I do not actually have access to any private or sensitive information.


In [19]:
def apply_guardrails(text):
    restricted_keywords = ['SSN', 'credit card', 'password']
    
    # Check for restricted content
    for keyword in restricted_keywords:
        if keyword.lower() in text.lower():
            raise ValueError("Generated content contains restricted information.")
    
    return text

# Example of applying guardrails to generated text
safe_text = apply_guardrails(generated_text)
print(f"Safe Text: {safe_text}")

Safe Text:  I'm an AI assistant created by Anthropic to be helpful, harmless, and honest. I do not actually have access to any private or sensitive information.


In [20]:
import json
import logging
from typing import Dict, Any

# Set up logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)

# Constants
HTTP_OK = 200
HTTP_BAD_REQUEST = 400

def lambda_handler(event: Dict[str, Any], context: Any) -> Dict[str, Any]:
    """
    Process generated text and apply content safety checks.
    
    :param event: The event dict containing the generated text
    :param context: The context object provided by AWS Lambda
    :return: A dictionary containing the status code and response body
    """
    logger.info("Received event: %s", event)
    
    try:
        # Input validation
        if 'generated_text' not in event:
            raise ValueError("Missing 'generated_text' in the event")
        
        generated_text = event['generated_text']
        
        # Apply guardrails
        safe_text = apply_guardrails(generated_text)
        
        response = {
            'statusCode': HTTP_OK,
            'body': json.dumps({
                'message': 'Content is safe.',
                'safe_text': safe_text
            })
        }
    except ValueError as e:
        logger.error("ValueError occurred: %s", str(e))
        response = {
            'statusCode': HTTP_BAD_REQUEST,
            'body': json.dumps({
                'message': str(e)
            })
        }
    except Exception as e:
        logger.error("Unexpected error occurred: %s", str(e))
        response = {
            'statusCode': HTTP_BAD_REQUEST,
            'body': json.dumps({
                'message': 'An unexpected error occurred.'
            })
        }
    
    logger.info("Returning response: %s", response)
    return response


In [21]:
# Example query for testing
user_query = "What is my SSN?"

# Prepare the request body
body = json.dumps({
    "prompt": f"\n\nHuman: {user_query}\n\nAssistant:",
    "max_tokens_to_sample": 50,
    "temperature": 0.7,
    "top_p": 1,
})

# Specify the model ID (replace with the appropriate model ID)
model_id = 'anthropic.claude-v2'  # or another appropriate model ID

# Invoke the model
response = bedrock_runtime.invoke_model(
    body=body,
    modelId=model_id,
    accept='application/json',
    contentType='application/json'
)

# Process the response
response_body = json.loads(response['body'].read())
generated_text = response_body['completion']

# Apply guardrails (you'll need to implement this function)
safe_text = apply_guardrails(generated_text)

print(f"Final Safe Response: {safe_text}")

Final Safe Response:  I do not actually have or store your Social Security number. I'm an AI assistant created by Anthropic to be helpful, harmless, and honest.
